<center><b>Assessed Coursework</b>

<p>In this iPython notebook, I will be looking at opinion extraction. Opinion extraction, also know as sentiment analysis is the process of using computing machines to derrive sentiment and opinions about various topics, including, although not limited to: product reviews, political policies or even public opinion on a sports team's perfomance.</p>
<p>Over the last few weeks I have completed work in the lab in which I have built an opinion extractor but incrementally adding functionality in the form of extensions. I will now show my code for my opinion extractor and the exetentions.</p>

In [ ]:
import sys
sys.path.append(r'T:\Departments\Informatics\LanguageEngineering')

In [70]:
import sys
sys.path.append(r'C:\Users\Jamie Mayer\Documents\NLE\LanguageEngineering')

The above code cells are used to import the NLTK into iPython. I have used the first one for working in the lab and the second one when working from home.

<center><b>Opinion Extractor</b>

In [71]:
# Main Opionion Extractor Function. The main function that a parsed sentence in passed in to
# along with an aspect token, e.g plot. Calls other functions in order to extract opinions 
# from the parsed sentence in relation to the aspect. Returns a list of opinions in the form of 
# strings.
def opinion_extractor(aspect_token, parsed_sentence):
    
    opinions = [] # List of strings that will ultimately be returned.
    
    opinions += get_amod(aspect_token, parsed_sentence) #calls the get_amod function that I added in Extension 1.
    
    opinions += get_copulae(aspect_token, parsed_sentence)
    
    opinions += get_conjunctions(aspect_token, parsed_sentence)

    return opinions


# get_amod function. Called inside the main opinion extractor function. Takes an aspect token and a parsed sentence.
# returns opinions, a list of strings.
def get_amod(aspect_token, parsed_sentence):
    opinions = [] #List of strings that the function will return.
    
    for dependant in parsed_sentence.get_dependants(aspect_token): # for every dependant of the aspect token:
        # if the dependant's relation on the aspect token is that of an adjectival modifier:
        if dependant.deprel == 'amod' and dependant.form != 'main' and dependant.form != 'special': 
            # makes a call to get_advmod to find if the dependant has an adverbial modifier and if it does
            # stores it in the variable 'adv'.
            adv = get_advmod(dependant, parsed_sentence)
            jj = dependant.form #the form of the dependant stored as a string.
            
            # makes a call to the get_neg function, passing in the aspect token and the parsed sentence
            # to find out if the opinion contains any negation.
            neg = get_neg(aspect_token, parsed_sentence)
            
            # if the call to get_advmod did not return an empty string:
            if adv != '':
                # concatinate 'neg' 'adv' and 'jj' into one string, hypenating 'adv and 'jj' and append it to 'opinions'
                opinions.append(neg + adv + '-' + jj)
            else:
                opinions.append(neg + jj) #concatinate 'neg' and 'jj' into one string and append it to 'opinions'
                 
    return opinions

#get_copulae function. Called inside main opinion extractor function. Takes an aspect token and a parsed sentence.
# Returns a list of copulae. List could be empty.

def get_copulae(aspect_token, parsed_sentence):
    copulae = [] #initialise list of copulae that will be returned.
    neg = '' # initialise the variable 'neg' to contain empty string.    
    # If the aspect token has dependency relation of 'nsubj' on it's head and the head is pos tagged as an adjective:
    if aspect_token.deprel == 'nsubj' and parsed_sentence.get_head(aspect_token).pos =='JJ':
            # Makes a call to get_advmod, passing the head of the aspect token and the parsed sentence to find if the
            # head of the aspect token has any adjectival modifiers.
            adv = get_advmod(parsed_sentence.get_head(aspect_token), parsed_sentence)            
            jj = parsed_sentence.get_head(aspect_token).form # stores the form of the head of the aspect token in 'jj'
            # Makes a call to get_neg, passing in the head of the aspect token and the parsed sentence. Checks if
            # the opinion contains any negation.
            neg = get_neg(parsed_sentence.get_head(aspect_token), parsed_sentence)
            # If 'adv' is not an empty string:
            if adv != '':
                    # Concatinate 'neg', 'adv' and 'jj' into one string, hypenating 'adv' and 'jj'
                    # and append it to 'copulae'
                    copulae.append(neg + adv + '-' + jj)
            else:
                    # Else concatinate 'neg' and 'jj' into one string and append to 'copulae'.
                    copulae.append(neg + jj)
    
    return copulae

# get_neg function. Takes a basic token object and a parsed sentence. Looks for negation in relation to the exressed
# opinion.
def get_neg(adj_token, parsed_sentence):            
    # for each dependant of 'adj_token':
    for dependant in parsed_sentence.get_dependants(adj_token):
        # if the dependant has a dependency relation of 'neg'
        if dependant.deprel == 'neg':
            return 'not ' # Return the string 'not'
    #else return the empty string
    return ''

# get_advmod function. Takes a basic token object and a parsed sentence. Looks for an adverbial modifier in relation to the
# expressed opinion
def get_advmod(adj_token, parsed_sentence):
    # for each token in the parsed sentence:
    for token in parsed_sentence:
        # if the dependency relation of the token is 'advmod' and the head is the token pased in:
        if token.deprel == 'advmod' and token.head == adj_token.id:
            #return that token.
            return token.form
    return ''

# get_conjunctions function. Takes a basic token object and a parsed sentence. Looks for conjuncts in parsed sentences.
def get_conjunctions(aspect_token, parsed_sentence):
    conjunctions = [] # Make an empty list called 'conjunctions'.
    # get the head of the token passed in. Get the dependants of said head. Then, for each token:
    for dependant in parsed_sentence.get_dependants(parsed_sentence.get_head(aspect_token)):
        # if the token has a dependency relation of 'conj':
        if dependant.deprel == 'conj':
            # call get_admov to see if the depenant has an adverbial modifier associtated with it and store the result in 
            # 'adv'
            adv = get_advmod(dependant, parsed_sentence)
            # call get_neg to find any negation that may be present in the opinion and store the result in 'neg'
            neg = get_neg(dependant, parsed_sentence)
            if adv != '': # Then if 'adv' does not contain the empty string:
                conjunctions.append(neg + adv + ' ' + dependant.form) # concatinate 'neg', 'adv' a single whitespace
                                                                      # character and the form of the dependant. And
                                                                      # and append it to 'conjunctions'
            else:
                # else if 'adv' does contain the empty strong then only append 'neg' and the form of the dependant and 
                # append it to the list 'conjunctions'
                conjunctions.append(neg + dependant.form)
    return conjunctions

The code in the next cell will load the example sentences test set used in the extentions from labs 8 & 9 and store them in a list, parsed_example_sentences.

In [72]:
from sussex_nltk.parse import load_parsed_example_sentences

parsed_example_sentences = load_parsed_example_sentences()

I will now demonstrate that my opinion extractor produces the correct output for the examples I have been given for each of the extensions. As the example sentences are all regarding the plot I will only be testing the extentions using the plot aspect for the example sentences test set.

<center><b>Extension 1: Adjectival Modification</b>

In extension 1 the objective was to tackle adjectival modification. I achevied this using the get_amod function. Of the 16 parsed sentences given in the example sentences testing set, the first 2 sentences are directly related to this extension and as such are the two sentences that I will focus on for this part of my report.

In [73]:
aspect = 'plot'
# for parsed sentences in parsed_example_sentences up to the index 1 (as we are only concerned with the first sentence
# for extension 1):
for parsed_sentence in parsed_example_sentences[:1]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	It	PRP	2	nsubj
2	has	VBZ	0	root
3	an	DT	6	det
4	exciting	JJ	6	amod
5	fresh	JJ	6	amod
6	plot	NN	2	dobj
7	.	.	2	punct
-------------------------------------
plot is ['exciting', 'fresh'] 




As the above code output shows, my opinion extractor clearly works for ajdectival modification. The opinion in the sentence states that 'It has an exciting, fresh plot.'


Therefore 'exciting' and 'fresh' should be extracted.

The fact that my extractor does this shows that it is functioning correctly for the example sentences test set for extension 1.

<center><b>Extension 2: Adjectives Linked by Copulae</b>

Extension 2 was all about finding adjectives linked to the aspect work via couplae. For example, if the opinion extractor only had Extension 1 functionality, then the parsed sentence 'the plot was dull.', would not return 'dull' as it relies on a copula. The second sentence in the example test set concerns copulae and so I will now put it through the opinion extractor to show that extension 2 is functioning as expected.

In [74]:
aspect = 'plot'
# for parsed sentences in parsed_example_sentences from index 1 up to index 2 (as we are only concerned with the second
# sentence for extension 2):
for parsed_sentence in parsed_example_sentences[1:2]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	plot	NN	4	nsubj
3	was	VBD	4	cop
4	dull	JJ	0	root
5	.	.	4	punct
-------------------------------------
plot is ['dull'] 




The output from the abouve code shows that for the second sentence in the example testing set, the opinion extractor is displaying the correct behavior. The token 'dull' has been extracted using the copula 'was'.

<center><b>Extension 3: Adverbial Modifiers</b>

In extension 3, we were to look at adverbial modifiers. This adresses situations where an adjective is modified by an adverb. When an adverb modifies a verb in a sentencem it will have the dependency relation of 'advmod'. The extension that I have added to the opinion extractor in this section will be able to extract adverbial modifiers and output them in the list of opinions. 

In [75]:
aspect = 'plot'
# for parsed sentences in parsed_example_sentences from index 2 up to index 4 (as we are only concerned with the second and
# third sentences for extension 3):
for parsed_sentence in parsed_example_sentences[2:4]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	It	PRP	2	nsubj
2	has	VBZ	0	root
3	an	DT	6	det
4	excessively	RB	5	advmod
5	dull	JJ	6	amod
6	plot	NN	2	dobj
7	.	.	2	punct
-------------------------------------
plot is ['excessively-dull'] 


1	The	DT	2	det
2	plot	NN	5	nsubj
3	was	VBD	5	cop
4	excessively	RB	5	advmod
5	dull	JJ	0	root
6	.	.	5	punct
-------------------------------------
plot is ['excessively-dull'] 




As can be seen in the output above. The token 'excessively' has an 'advmod' relation with it's head being the adjective 'dull' which in turn is dependant on the aspect 'plot'. The output 'excessively-dull'. This shows that my opinion extractor is working for the third sentence in the example testing set.

<center><b>Extension 4: Negation</b>

The extensions already discussed are unable to cope with any form of negation. For example if the parsed sentence, 'the plot wasn't dull' was passed into the opinion extractor with only extensions 1-3 implemented, then the opinion 'dull' would be extracted, which is entirely inaccurate. This is because the sentence contains negation. The code below will run the 4th sentence through the opinion extractor.

In [76]:
aspect = 'plot'
# for parsed sentences in parsed_example_sentences from index 4 up to index 7.
for parsed_sentence in parsed_example_sentences[4:7]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	plot	NN	5	nsubj
3	was	VBD	5	cop
4	n't	RB	5	neg
5	dull	JJ	0	root
6	.	.	5	punct
-------------------------------------
plot is ['not dull'] 


1	It	PRP	7	nsubj
2	was	VBD	7	cop
3	n't	RB	7	neg
4	an	DT	7	det
5	exciting	JJ	7	amod
6	fresh	JJ	7	amod
7	plot	NN	0	root
8	.	.	7	punct
-------------------------------------
plot is ['not exciting', 'not fresh'] 


1	The	DT	2	det
2	plot	NN	6	nsubj
3	was	VBD	6	cop
4	n't	RB	6	neg
5	excessively	RB	6	advmod
6	dull	JJ	0	root
7	.	.	6	punct
-------------------------------------
plot is ['not excessively-dull'] 




My negation function has taken the sentence and correctly identified that the opinion is that the plot was not dull. If the function identies negation will add the word 'not' to the front of the adjective. In this case 'not dull'.

<center><b>Extension 5: Conjunction</b>

A conjunction is used to connect words in a sentence. For example in the sentence 'The plot was cheesy, but fun and inspiring.', 'fun' and 'cheesy' are both conjoined with 'cheesy' which istelf is dependent on the aspect 'plot'. This means that they both also apply to the aspect. By adding extension 5, I hope to add the functionality to my opinion extractor, making it able to extract conjuncts as a part of the opinion.

In [77]:
aspect = 'plot'
# for parsed sentences in parsed_example_sentences from index 7 up to index 9. 

for parsed_sentence in parsed_example_sentences[7:9]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	plot	NN	4	nsubj
3	was	VBD	4	cop
4	cheesy	JJ	0	root
5	,	,	4	punct
6	but	CC	4	cc
7	fun	NN	4	conj
8	and	CC	4	cc
9	inspiring	VBG	4	conj
10	.	.	4	punct
-------------------------------------
plot is ['cheesy', 'fun', 'inspiring'] 


1	The	DT	2	det
2	plot	NN	5	nsubj
3	was	VBD	5	cop
4	really	RB	5	advmod
5	cheesy	JJ	0	root
6	and	CC	5	cc
7	not	RB	9	neg
8	particularly	RB	9	advmod
9	special	JJ	5	conj
10	.	.	5	punct
-------------------------------------
plot is ['really-cheesy', 'not particularly special'] 




The output of the code shows that, when applying my opinion extractor to the testing set of sentences provided, extension 5 has been implemented successfully. Further evidence of the negation function displaying the correct behavoir can also be seen in the second sentence by successfully extracting the opinion of the plot being 'not particularly special'.

I have shown that extensions 1 to 5 are working and behaving as expected, however my opinion extractor does have some limitations which, had I implemented the additional extensions. I will now run the final example sentences from the testing set through my opinion extractor to highlight the limitations.

In [78]:
aspect = 'plot'
# for parsed sentences in parsed_example_sentences from index 7 up to index 9. 

for parsed_sentence in parsed_example_sentences[9:]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	script	NN	7	nsubj
3	and	CC	2	cc
4	plot	NN	2	conj
5	are	VBP	7	cop
6	utterly	RB	7	advmod
7	excellent	JJ	0	root
8	.	.	7	punct
-------------------------------------
plot is ['plot'] 


1	The	DT	2	det
2	script	NN	6	nsubj
3	and	CC	2	cc
4	plot	NN	2	conj
5	were	VBD	6	cop
6	unoriginal	JJ	0	root
7	and	CC	6	cc
8	boring	VBG	6	conj
9	.	.	6	punct
-------------------------------------
plot is ['plot'] 


1	The	DT	2	det
2	plot	NN	5	nsubj
3	was	VBD	5	aux
4	n't	RB	5	neg
5	lacking	VBG	0	root
6	.	.	5	punct
-------------------------------------
plot is [] 


1	The	DT	2	det
2	plot	NN	4	nsubj
3	is	VBZ	4	cop
4	full	JJ	0	root
5	of	IN	4	prep
6	holes	NNS	5	pobj
7	.	.	4	punct
-------------------------------------
plot is ['full'] 


1	There	EX	2	expl
2	was	VBD	0	root
3	no	DT	5	det
4	logical	JJ	5	amod
5	plot	NN	2	nsubj
6	to	TO	5	prep
7	this	DT	8	det
8	story	NN	6	pobj
9	.	.	2	punct
-------------------------------------
plot is ['logical'] 


1	I	PRP	2	nsubj
2	loved	VBD	0	root
3	the	DT	4	det
4	plot	

In the first two examples, the aspect 'plot' is conjunct on another aspect 'script'. Although I have addressed conjunction in Extension 5, it has been implemented with the intent of being applied to adjectives dependent on the aspect rather than the aspect it self being a conjunction.

<center><b> Applying Opinion Extractor to Amazon DVD Corpus</b>

I will now attempt to assess the limitations of my opinion extractor when applied to the sentences in the corpus of Amazon DVD reviews that I have been provided with.

One of the parameters that my opinion extractor takes is a parsed sentence, which means that the sentences I am going to analyse must first be run through a parser before they can be passed to the opinion extractor. This is turn means that they must be Part of Speech (POS) tagged in order to be parsed. 

Before I do either of these things I will first go into more depth about what they are and what they involve. First I will discuss POS tagging.

<b> Part of Speech Tagging</b>


POS tagging is the proccess of categorising words according to their gramatical function. These can fall into one of many categories, the main ones of which being:


                                            Adjective
                                            Noun
                                            Verb
                                            Adverb
                                            Conjunction
                                            Pronoun
                                            Interjection
                                            Preposition
                                            Determiner
                                     

POS tags are not universal and there exist differing sets of POS tags, for example the Penn Tree bank contains 45 tags, the Brown corpus contains 87 and the C7 tagset contains 146. Jurafsky and Martin (2009) explain that the added granularity that comes with a greater number of tags can be rather useful, using the example of distinguishing between possessive pronouns and personal pronouns they say that knowing this information is a key factor in predicting other words that are likely to crop up around the word in question.

POS tagging serves multiple purposes, mainly those of (i) disambiguating words, (ii) prediction of what words may preceed or follow a certain word and (iii) for helping to spot gramatical structure in the bigger picture, this last purpose is very related to parsing which I shall discuss in due course. 

POS tags can be split up into 2 classes, these are the closed classes and the open classes. Closed classes get their name from the fact that there is a fixed set of them, implying that they are very rarely (if ever), added to. They do not convey any meaning or posses any content but are of vital importance gramatically as they form the backbone of the main structure of the sentence. Words like 'the', 'by' and 'and' belong to a closed POS classes (article, preposition and conjunction respectively). They occur frequently and are usually relatively short in length. In contrast open classses are, as their name suggests more open in that they are added to frequently. The open classes contain the main content and will generally be associated with the actual meaning of the sentence. Another interesting point raised by Jurafsky and Martin in the same text is that there are four major open word classes in world languages;

                                            Nouns
                                            Verbs
                                            Adjectives
                                            Adverbs

The interesting thing that they remark upon is that English contains all of these classes, however not all languages do. As mentioned previously, these classes can be further categorised into sub classes, for example a Noun could be of the class, porper noun, common noun, mass noun etc. 
                                            

So when a sentence is POS tagged, each word in that sentence is assigned its own tag, e.g adjective. Although this is not an easy task as one word could have high ambiguity, meaning that a single word could have different meanings and thus a different POS tag depending on its context. For example the word 'milk' could be a noun or a verb. In the sentence 'Would you like milk with that?' it is easy for a human to see that milk is being used in the context of a noun. However the same word is used in the sentence 'Don't milk it' where it is used as an adjective. Often an ambigious word is a lot more likely to have one tag than another, for example a word could occur as a noun 98% of the time while the other 2% of the time it could appear as an adjective. 

A POS tag often has an abrevation associated with it, i.e a noun could be 'NN', an adjective 'JJ', a determiner 'DET' etc. Once a sentence has been POS tagged it can then be dependency parsed. 



<b>Dependency Parsing</b>


Dependency parsing is type of automated analysis of syntax that relies upon the idea that words are related to each other by directed links called dependencies. If word 'a' is dependant on word 'b' then 'a' is said to be the dependant and 'b' is said to be the head of that dependant. Dependancy Parsing uses binary relations, that is to say a single relation only occurs between 2 words. However one word can have multiple dependants but must always only have one head. During the parsing, an artificial token as added called 'root'. Kübler,S et al. (2009), explain how this is done in order to allow us to form tighter formal definitions, in that we can say all tokens must always have a head, even if that head is an artificial word that has been added and bears no meaning, context or even adds anything to the sentence at all. The attribute of dependency relation describes the type of relationship between the head and its dependant. For example a relation of 'advmod' describes an adverbial modification relation type, where the dependant is an adverb that modifies its head. When applied to the entire sentence, a structure of dependency/head relations is formed, an example of which can be seen below.




![](./images/Capture3.jpg)
<center>Kübler,S et al. (2009)</center>

<br> The example illustrates a parsed sentence's dependency structure. Taking the word 'had' as an example, it's head is 'root' and it has 3 dependants, one of which is the word 'news'. News has a dependency type 'subject' and in is the head of the word 'economic' which has a dependency relation of attribute. The above dependency tree illustrates how a dependency structure of a sentence visually, with each word having a head. The arrows always go from head to dependant

<br><br>When a sentence has been dependency parsed each word is labeled with the following attributes;

                                            ID -  A unique identifer local to the sentence
                                            
                                            Form - A string containing the actual word
                                            
                                            POS tag - The POS tag of the word
                                            
                                            Head - The head in that word's dependency relation
                                            
                                            Dependency Relation - The relation type of the word to its head
                                            
When a sentence has been parsed, it can then be used in the opinion extractor.

I will now run sample sets of reviews from the Amazon DVD corpus through my opinion extractor.

In [145]:
from sussex_nltk.parse import load_parsed_dvd_sentences

parsed_sentences_plot = load_parsed_dvd_sentences('plot')

parsed_sentences_characters = load_parsed_dvd_sentences('characters')

parsed_sentences_cinematography = load_parsed_dvd_sentences('cinematography')

parsed_sentences_dialogue = load_parsed_dvd_sentences('dialogue')


In [179]:
aspect = 'plot'
for parsed_sentence in parsed_sentences_plot[:9]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	plot	NN	5	nsubj
3	was	VBD	5	cop
4	less	JJR	5	amod
5	coherent	NN	0	root
6	,	,	5	punct
7	but	CC	5	cc
8	if	IN	10	mark
9	you	PRP	10	nsubj
10	watch	VBP	21	advcl
11	flicks	NNS	10	dobj
12	like	IN	10	prep
13	this	DT	12	pobj
14	for	IN	13	prep
15	the	DT	16	det
16	plot	NN	14	pobj
17	,	,	21	punct
18	something	NN	21	nsubj
19	is	VBZ	21	cop
20	clinically	RB	21	advmod
21	wrong	JJ	5	conj
22	with	IN	21	prep
23	you	PRP	22	pobj
24	.	.	5	punct
-------------------------------------
plot is ['clinically wrong'] 


1	The	DT	2	det
2	plot	NN	5	nsubj
3	was	VBD	5	cop
4	less	JJR	5	amod
5	coherent	NN	0	root
6	,	,	5	punct
7	but	CC	5	cc
8	if	IN	10	mark
9	you	PRP	10	nsubj
10	watch	VBP	21	advcl
11	flicks	NNS	10	dobj
12	like	IN	10	prep
13	this	DT	12	pobj
14	for	IN	13	prep
15	the	DT	16	det
16	plot	NN	14	pobj
17	,	,	21	punct
18	something	NN	21	nsubj
19	is	VBZ	21	cop
20	clinically	RB	21	advmod
21	wrong	JJ	5	conj
22	with	IN	21	prep
23	you	PRP	22	pobj
24	.	.	5	punct
-------------------------------------
plot

In [178]:
aspect = 'characters'

for parsed_sentence in parsed_sentences_characters[:9]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s are %s \n\n' % (aspect, opinion) #print 'aspect are opinion'

1	The	DT	3	det
2	main	JJ	3	amod
3	characters	NNS	16	nsubj
4	in	IN	3	prep
5	Divided	NNP	7	nn
6	We	NNP	7	nn
7	Fall	NNP	4	pobj
8	,	,	7	punct
9	Josef	NNP	7	conj
10	and	CC	7	cc
11	Maria	NNP	7	conj
12	,	,	3	punct
13	are	VBP	16	cop
14	a	DT	16	det
15	married	JJ	16	amod
16	couple	NN	0	root
17	who	WP	18	nsubj
18	end	NN	16	rcmod
19	up	RP	18	prt
20	harboring	VBG	18	xcomp
21	a	DT	23	det
22	Jewish	JJ	23	amod
23	acquaintance	NN	20	dobj
24	in	IN	23	prep
25	their	PRP$	26	poss
26	cellar	JJ	24	pobj
27	for	IN	20	prep
28	three	CD	29	num
29	years	NNS	27	pobj
30	.	.	16	punct
-------------------------------------
characters are [] 


1	With	IN	13	prep
2	a	DT	3	det
3	cast	JJ	1	pobj
4	as	IN	13	quantmod
5	large	JJ	13	advcl
6	as	IN	5	prep
7	this	DT	8	det
8	one	CD	6	pobj
9	,	,	13	punct
10	it	PRP	13	nsubj
11	's	VBZ	13	cop
12	only	RB	13	advmod
13	natural	JJ	0	root
14	that	IN	18	complm
15	short	JJ	16	amod
16	thrift	NN	18	nsubjpass
17	be	VB	18	auxpass
18	given	VBN	13	ccomp
19	to	TO	18	prep
20	peripheral	JJ	21	amod
21	

In [176]:
aspect = 'cinematography'

for parsed_sentence in parsed_sentences_cinematography[:9]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	plot	NN	4	nsubj
3	is	VBZ	4	aux
4	uninvolving	VBG	0	root
5	and	CC	4	cc
6	incoherent	NN	11	nsubjpass
7	,	,	11	punct
8	the	DT	9	det
9	cinematography	NN	11	nsubjpass
10	is	VBZ	11	auxpass
11	cramped	VBN	4	conj
12	and	CC	11	cc
13	completely	RB	14	advmod
14	lacking	VBG	11	conj
15	in	IN	14	prep
16	style	NN	15	pobj
17	,	,	21	punct
18	the	DT	19	det
19	music	NN	21	nsubj
20	is	VBZ	21	cop
21	weak	JJ	4	dep
22	during	IN	4	prep
23	a	DT	24	det
24	time	NN	22	pobj
25	that	WDT	26	nsubj
26	spawned	VBD	24	rcmod
27	so	RB	28	advmod
28	many	JJ	30	amod
29	memorable	JJ	30	amod
30	soundtracks	NNS	26	dobj
31	and	CC	30	cc
32	the	DT	33	det
33	direction	NN	30	conj
34	lacks	VBZ	4	aux
35	any	DT	37	det
36	focus	NN	37	nn
37	whatsoever	NN	4	dobj
38	.	.	4	punct
-------------------------------------
cinematography is ['completely lacking'] 


1	The	DT	2	det
2	sets	NNS	5	nsubj
3	are	VBP	5	aux
4	quite	RB	5	advmod
5	amazing	VBG	0	root
6	,	,	5	punct
7	and	CC	5	cc
8	the	DT	9	det
9	cinematography	NN	12	nsubj
10	i

In [177]:
aspect = 'dialogue'
for parsed_sentence in parsed_sentences_dialogue[:9]:
    # for each occurence of the aspect token in the sentence:
    for aspect_token in parsed_sentence.get_query_tokens(aspect):
        
        opinion = opinion_extractor(aspect_token, parsed_sentence) # call opinion_extractor and store result in 'opinion'.
        print parsed_sentence # print the entire parsed sentence.
        print '-------------------------------------' # separator
        print '%s is %s \n\n' % (aspect, opinion) #print 'aspect is opinion'

1	The	DT	2	det
2	performances	NNS	4	nsubj
3	are	VBP	4	cop
4	thin	JJ	0	root
5	and	CC	4	cc
6	uneven	JJ	4	conj
7	,	,	4	punct
8	the	DT	9	det
9	plot	NN	11	nsubj
10	is	VBZ	11	cop
11	inconsistent	NN	4	conj
12	,	,	11	punct
13	and	CC	11	cc
14	the	DT	15	det
15	dialogue	NN	16	nsubj
16	sounds	VBZ	11	conj
17	like	IN	16	prep
18	it	PRP	17	pobj
19	was	VBD	20	auxpass
20	pulled	VBN	4	dep
21	from	IN	20	prep
22	fortune	NN	23	nn
23	cookies	NNS	21	pobj
24	made	VBN	23	partmod
25	by	IN	24	prep
26	the	DT	27	det
27	cast	JJ	25	pobj
28	of	IN	27	prep
29	Friends	NNPS	28	pobj
30	.	.	4	punct
-------------------------------------
dialogue is [] 


1	Her	PRP$	2	poss
2	dialogue	NN	4	nsubjpass
3	is	VBZ	4	auxpass
4	kept	VBN	0	root
5	short	JJ	4	acomp
6	and	CC	5	cc
7	simple	JJ	5	conj
8	,	,	4	punct
9	and	CC	4	cc
10	when	WRB	13	advmod
11	she	PRP	13	nsubj
12	does	VBZ	13	aux
13	have	VB	33	advcl
14	more	JJR	15	mwe
15	than	IN	16	quantmod
16	3	CD	17	num
17	lines	NNS	13	dobj
18	at	IN	13	advmod
19	once	RB	18	advmod
20	,	,	33	punct
2

Looking at the output of the code above it appears that my opinion extractor performs extremely poorly when applied to sentences in the Amazon DVD corpus. The opinion extractor performed as follows for the respective aspects:

                                    Plot 2/10 opinions extracted correctly
                                    
                                    Characters 2/9 opinions extracted correctly
                                    
                                    cinematography 4/9 opinion extracted correctly
                                    
                                    dialogue 2/9 opinions extracted correctly

I belive that the poor performance of my opinion extractor when using samples from the amazon DVD corpus compared to the relative success when using the example test set, is down to a few reasons. Mainly I think that this is a direct result of tailoring my extensions to work well with the example testing set. However in the wider corpus there is a much greater variation on how sentences can be structured while still conveying the same sentiment and opinions. For example the sentences 'The plot made watching this movie a complete waste of time.' and 'Watching this movie was a complete waste of time because of the plot' both mean the same thing but are structured in different ways. The other reason that I think may be responsible for my opinion extractor failing is closely linked with the last reason in that the testing set was provided with the intent of being used for the extensions in labs 8 and 9. I feel that had I implemented the extra extensions, my opinion extractor would have performed better as these address more complex gramatical structure of sentences. For example while my extractor function has been show to correctly extract the opinion 'not dull' from 'the plot wasn't dull' it would fall down for 'I think the plot wasn't dull'. But with added functionality in the form of further extensions, this could be achieved. While it is possible that subset of the errors are down to POS tagger errors or parser errors, I feel that the moajority of the blame lies with the opinion extractor.

<center><b> Website Proposal</b>

<p>In this section I will put forward a proposal as to how I would go about creating a website that automatically produces summaries of what people think about various aspects of a DVD.</p><br>
<p>Firstly from the retailer's point of view, implementing NLP techniques such as those I have discussed in this report would allow them to profile their users based on what aspects of a DVD they view as important. This could be done by enforcing a policy that in order to leave a review on a product, the reviewer must be a registered member of the site. This would allow all reviews to be assiciated with one particular person. The reviews could then be run through NLE software which extracts information about which aspects the reviewer talks about the most in their reviews. All DVDs could then have their reviews also run through similar NLP software that extracts the total number of positive reviews for each aspect. The user could then be matched up with DVDs which have a high number of positive reviews mentioning the aspects that they consider most valuable based on their previous reviews that they have themselves written. These DVDs could be presented to the reviewer on page shown after their initial log in. This would be a mutually beneficial process to implement as the reviewer is presented with DVDs which have good reviews on aspects that they deem important and as a result of this the retailer would see an increase in revenue, supposing that good software was implemented and the reviewer has faith in the system providing accurate suggestions.</p><br>
<p>Another way that NLP software could be used in an online DVD retail business involves the retailer and the studio that makes the movies. Again the motivation behind this would be a monetary one. The retailer could implement an opinion extractor tailored specifically to DVDs, and in a simalar fashion to the previous idea take a count of which aspect is spoken about the most but the key difference is to also take negative reviews into account as well as positive reviews. This information would be extremely valuable to the film studios who may or may not consider making sequels or other movies that fall into the same genre to gauge what the target audience deems important fot that type of movie. This would allow them to make improvements to areas that are found to be lacking by reviewers and also to see what they are doing right that they may want to keep doing in the future. The retailer could charge the studios a fee for processing and providing this information, therefore again increasing revenue.</p><br>
<p>As with any form of commercial software implementation problems may arise. Accuracy stands out as the biggest hurdle in this situation, as POS tagging and Parsing are not trivial tasks and errors do occur. If a sentence has been incorrectly POS tagged (for example a word with high ambiguity is tagged as one pos class when in the context concerned it should be tagged as another) then it will be incorrectly parsed and then as an effect of that will not have the correct opinion extracted from it. The challenge is to ensure that tagging and parsing errors are minimised so that the opinion extractor has the best chance of being accurate. Also the opinion extractor itself needs to be robust enough to cope with the variety of gramatical structure present in the language it is being applied to.</p>
<p>This leads on to my final point that is people writing online reviews for products may not have the best literacy skills leading to human errors in the way of grammar, spelling and punctuation, leading to the errors mentioned above. One way around this would be to implement some sort of algortithm that holds common typing, spelling, grammar and punctiation errors and checks each word when it is being tokenised. This however would undoubtably involve an high computational cost unless some sort of dynamic approach similar to a sequence alignment algorithm.</p>

<b> References </b>

Jurafsky, Dan, and James H Martin. Speech And Language Processing. Upper Saddle River, N.J.: Pearson Prentice Hall, 2009. Print.

Kübler, Sandra, Ryan McDonald, and Joakim Nivre. Dependency Parsing. [San Rafael, Calif.]: Morgan & Claypool Publishers, 2009. Print.